In [1]:
import sys, os
import pandas as pd
sys.path.append("../../../../")

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as influx_Client
from Clust.clust.ingestion.mongo.mongo_client import MongoClient
db_client = influx_Client.InfluxClient(ins.CLUSTDataServer2)
mongo_client = MongoClient(ins.CLUSTMetaInfo2)

import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

from Clust.clust.ML.common import train_pipeline

cuda is available.


## 1. Data Preparation

### 1-1. Parameter Setting

In [2]:
app_name= "Hs2SwineFarmWithWeatherTime" # "Hs2SwineFarmWithWeatherTime", "energy"

if app_name == "energy":
    model_purpose = 'regression'
    feature_X_list = ['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7',
       'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7',
       'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']
    feature_y_list = ['value']
    split_mode ="window_split"
    data_y_flag = True # 이미 만들어진 Y 데이터를 활용함
    
elif app_name == "Hs2SwineFarmWithWeatherTime":
    model_purpose = 'forecasting' 
    feature_X_list = ['Temperature', 'out_temp','sin_hour']
    feature_y_list = ['Temperature']
    split_mode = 'step_split'
    data_y_flag = False # Y데이터는 없음, X 에서 Y 데이터를 도출함
    
step = 'train'
bucket_name = 'integration' 
scaler_path = './scaler/'

data_clean_level = 0 
dataset_name = model_purpose + '_' + app_name  
data_name_X = dataset_name + '_cleanLevel' + str(data_clean_level)+'_'+step+'X'
data_name_y = dataset_name+'_cleanLevel' + str(data_clean_level)+'_'+ step+'y'
data_meta = mongo_client.get_document_by_json('integration', dataset_name, {'data_name':data_name_X})[0]
all_integrated_ms_list = db_client.measurement_list(bucket_name)
print(all_integrated_ms_list)
collection_list = mongo_client.get_collection_list(bucket_name)
print(collection_list)

['classification_actionPattern_testX_cleanLevel0', 'classification_actionPattern_testy_cleanLevel0', 'classification_actionPattern_trainX_cleanLevel0', 'classification_actionPattern_trainy_cleanLevel0', 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel0_test', 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel0_testX', 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel0_train', 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel0_trainX', 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_test', 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_testX', 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_train', 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainX', 'forecasting_Hs2SwineFarmWithWeatherTime_test_cleanLevel0', 'forecasting_Hs2SwineFarmWithWeatherTime_test_cleanLevel0_X', 'forecasting_Hs2SwineFarmWithWeatherTime_test_cleanLevel4', 'forecasting_Hs2SwineFarmWithWeatherTime_test_cleanLevel4_X', 'forecasting_Hs2SwineFarmWithWeatherTime_train_cleanLevel0', 'for

### 1-2. Data Ingestion
#### 1-2-1. X-y Data Ingestion pipeline

In [3]:
# Data Ingestion
ingestion_method = 'ms_all'
ingestion_param_X = {
    "bucket_name" : bucket_name,
    'ms_name' : data_name_X,
    'feature_list' : feature_X_list                              
}
ingestion_param_y = {
    "bucket_name" : bucket_name,
    'ms_name' : data_name_y,
    'feature_list' : feature_y_list                              
}

In [4]:
data_X, data_y = train_pipeline.get_Xy_data_in_Clust(ingestion_param_X, data_y_flag, ingestion_param_y, ingestion_method, db_client)

### 1-2-2. Random Nan Insert (Test)

In [5]:
nan_ratio = 0.005

In [6]:
data_X = train_pipeline.random_nan_df(data_X, nan_ratio)
data_y = train_pipeline.random_nan_df(data_y, nan_ratio)

../../../../Clust/clust/ML/common/train_pipeline.py:31: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  df.loc[df.sample(frac=nan_ratio). index, col] = pd.np.nan
/home/jwmoon/.conda/envs/sejong/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/home/jwmoon/.conda/envs/sejong/lib/python3.8/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


#### 1-2-3. Data scaling

In [7]:
scaler_param='scale'
scale_method='minmax'

In [8]:
dataX_scaled, X_scalerFilePath, datay_scaled, y_scalerFilePath= train_pipeline.get_scaled_data_in_Clust(data_name_X, data_X, data_name_y, data_y, scaler_path, scaler_param, scale_method)

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


['Temperature', 'out_temp', 'sin_hour']
Make New scaler File
['Temperature']
Make New scaler File


## 2. Cleaning and split
### 2.1 Cleaning1 pipeline

In [9]:
model_clean = True
nan_process_info = {'type':'num', 'ConsecutiveNanLimit':10000, 'totalNaNLimit':10000}

In [10]:
dataX_scaled = train_pipeline.get_clean_model_data_1(model_clean, nan_process_info, dataX_scaled)

3 ---> 3


### 2.2 Train/Val Split pipeline

In [11]:
split_ratio = 0.8

In [12]:
# TODO 데이터 나뉘는 부분 추가로 작성된 것 지수님에게 물어봐야 함
day_window_size = train_pipeline.get_default_day_window_size(dataX_scaled)
train_x, val_x, train_y, val_y = train_pipeline.get_split_data_in_CLUST(split_mode, split_ratio, dataX_scaled, datay_scaled, day_window_size)

### 2.3 Data Transformation & Clean2 pipeline

In [13]:
max_nan_limit_ratio = 0.5
if split_mode =='window_split':
    transformParameter = {
            'past_step':day_window_size,
            'max_nan_limit_ratio': max_nan_limit_ratio
    }
else:
    transformParameter = {
            'future_step': 2,
            'past_step': 24, 
            'max_nan_limit_ratio': max_nan_limit_ratio
    }

In [14]:
train_X_array, train_y_array = train_pipeline.get_transfomed_data(split_mode, transformParameter, train_x, train_y)
val_X_array, val_y_array = train_pipeline.get_transfomed_data(split_mode, transformParameter, val_x, val_y)

nan_limit_num:  12
Original num: 8516 Final num: 8310 NaN num: 206
nan_limit_num:  12
Original num: 2129 Final num: 2092 NaN num: 37


### 2.4 Set Model and train parameter

In [15]:
# RNN models (RNN, LSTM, GRU) parameters
seq_len, input_size = train_X_array.shape[1], train_X_array.shape[2]
model_method = 'GRU_rg' # Set model methods i.e., 'LSTM_rg', 'GRU_rg', 'CNN_1D_rg', 'LSTM_FCNs_rg', 'FC_rg' 
if model_method == 'LSTM_rg' or model_method == 'GRU_rg':
    modelParameter = {
        'rnn_type': 'lstm',
        'input_size': input_size, 
        'hidden_size': 64,
        'num_layers': 2,
        'output_dim': 1, 
        'dropout': 0.1, 
        'bidirectional': True
    }
# CNN_1D model parameters
elif model_method == 'CNN_1D_rg':
    modelParameter = {
    'input_size': input_size,
    'seq_len': seq_len,
    'output_channels': 64,
    'kernel_size': 3,
    'stride': 1,
    'padding': 0, 
    'dropout': 0.1
    }
# LSTM_FCNs model parameters
elif model_method == 'LSTM_FCNs_rg':
    modelParameter = {
    'input_size': input_size,
    'num_layers': 2,
    'lstm_dropout': 0.4,
    'fc_dropout': 0.1
    }
# FC model parameters
elif model_method == 'FC_rg':
    modelParameter = {
    'input_size': input_size,
    'dropout': 0.1,
    'bias': True
    }
    
train_parameter = {
    'lr': 1e-4,
    'weight_decay': 1e-6, 
    'device': 'cpu', 
    'n_epochs': 10, 
    'batch_size': 16
}
model_name = None
model_file_path = None

### 2.5 Set Model name and path pipeline

In [16]:
# model_name and file path check and create
model_name = train_pipeline.get_default_model_name(model_name, app_name, model_method, model_clean)
model_file_path = train_pipeline.get_default_model_path(model_name, data_name_X, model_method, train_parameter)

./Models/GRU_rg/Hs2SwineFarmWithWeatherTime_GRU_rg_True/forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel0_trainX/d531eec11664669cff1f6a3ad9639012/model.pkl


## 3. Training

In [ ]:
from Clust.clust.ML.regression_YK.train import RegressionTrain as RML

rml = RML()
rml.set_param(train_parameter)
rml.set_model(model_method, modelParameter)
rml.set_data(train_X_array, train_y_array, val_X_array, val_y_array, )
rml.train()
rml.save_best_model(model_file_path)

Start training model
[1/10] Training loss: 0.0541	 Validation loss: 0.0013


## 4. save metadata

In [ ]:
# from Clust.clust.transformation.general.dataScaler import encode_hash_style
modelTags =["model_tag_example"]
trainDataType = "timeseries"

modelInfoMeta ={
    "trainDataInfo":data_meta,
    "modelName":model_name,
    "featureList":feature_X_list,
    "target": feature_y_list,
    "trainDataType":trainDataType,
    "modelPurpose":model_purpose,
    "modelMethod":model_method,
    "modelTags":modelTags,
    "modelCleanLevel":model_clean,
    "trainParameter": train_parameter,
    "modelParameter": modelParameter,
    "transformParameter":transformParameter,
    "scalerParam":scaler_param,
    "trainDataName":[data_name_X, data_name_y], 

    "files":{
        "modelFile":{
            "fileName":"model.pth",
            "filePath":model_file_path
        },
        "XScalerFile":{
            "fileName":"scaler.pkl",
            "filePath":X_scalerFilePath       
        },
        "yScalerFile":{
            "fileName":"scaler.pkl",
            "filePath":y_scalerFilePath      
        }
    }
}

In [ ]:
from Clust.clust.ML.tool import meta as ml_meta
modelInfoMeta = ml_meta.save_model_meta_data(mongo_client, modelInfoMeta)